In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from data import ModelNet40,download,load_data
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from matplotlib import pyplot as plt
import pickle
import warnings
import math
import time
from datetime import datetime
from collections import OrderedDict
from scipy.optimize import linear_sum_assignment
from torch import jit
%matplotlib inline
import os
from torch.utils.data import Dataset, TensorDataset

In [14]:
train_losses = []
test_results_exp = []
test_accs = []

In [16]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lrt = lr * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lrt

In [2]:
device = torch.device("cuda:2")

In [36]:
data = ModelNet40(num_points=2048, partition='train', gaussian_noise=False,
                       unseen=False, factor=4)
data_test = ModelNet40(num_points=2048, partition='test', gaussian_noise=False,
                       unseen=False, factor=4)
train_loader = DataLoader(data, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(data_test, batch_size=1, shuffle=True, drop_last=True)

/home/gwu/ConvexICP/data.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)


In [39]:
def train_eval_icp(model,device, train_loader,test_loader, optimizer,criterion, epoch, eval_mode='on'):
    global best
    model.train()
    final_cal = nn.Sigmoid()
    train_error_logs = []
    t1 = datetime.now()
    test_loss_log = []
    test_acc_log = []
    train_corrects = 0
    
    for batch_idx, (src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba) in enumerate(train_loader):
        src = src.to(device)
        target = target.to(device)
        rotation_ab = rotation_ab.to(device)
        translation_ab = translation_ab.to(device)
        rotation_ba = rotation_ba.to(device)
        translation_ba = translation_ba.to(device)
        optimizer.zero_grad()
        output_train = model(src,target,rotation_ab,translation_ab)
        loss = criterion(output_train, target)
        loss.backward()
        optimizer.step()
        #pred_train = output_train.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        #train_corrects += pred_train.eq(target.view_as(pred_train)).sum().item()
        train_error_logs.append(loss.item())
        
        if (batch_idx+0) % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}'.format(
                epoch, (batch_idx+0)* len(target), len(train_loader.dataset),
                100. * (batch_idx+0) / len(train_loader), loss,(datetime.now()-t1).total_seconds()))

        
    print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%)\n'.format(
                epoch, train_corrects, len(train_loader.dataset),
                100. * train_corrects / len(train_loader.dataset)))
    
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba in enumerate(test_loader):
            src = src.to(device)
            target = target.to(device)
            rotation_ab = rotation_ab.to(device)
            translation_ab = translation_ab.to(device)
            rotation_ba = rotation_ba.to(device)
            translation_ba = translation_ba.to(device)
            output_test = model(src,target,rotation_ab,translation_ab)
            test_loss += criterion(output_test, target_test).item()  # sum up batch loss
    test_loss /= len(test_loader.dataset)
    test_loss_log.append(test_loss)
    print('Test set: Average loss: {:.8f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct_test, len(test_loader.dataset),
            100. * correct_test / len(test_loader.dataset)))
    return train_error_logs,test_loss_log,test_acc_log

In [5]:
class PointNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv3 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=1, bias=False)
        self.conv5 = nn.Conv1d(128, emb_dims, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(emb_dims)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        return x

In [6]:
class CPNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(CPNet, self).__init__()
        self.base_net = PointNet()
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2, R, T):
        
        trans = torch.matmul(R, x1) + T.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)

        return dist

In [32]:
torch.Tensor(1,3,3)

tensor([[[ 1.3452e-43,  0.0000e+00,  8.9683e-44],
         [ 0.0000e+00, -1.1594e+09,  3.0635e-41],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]])

In [35]:
class CPEvalNet(nn.Module):
    def __init__(self, base_net, emb_dims=512):
        super(CPEvalNet, self).__init__()
        self.base_net = PointNet()
        self.base_net.load_state_dict(base_net.state_dict())
        for param in self.base_net:
            param.require_grad = False
        self.rotation = nn.Parameter(torch.Tensor(1,3,3))
        self.translation = nn.Parameter(torch.Tensor(1,3))
        nn.init.kaiming_uniform_(self.rotation, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.translation, a=math.sqrt(5))
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2):
        
        trans = torch.matmul(self.rotation, x1) + self.translation.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)

        return dist

In [7]:
def dummy_loss(output, target):
    loss = torch.mean(output)
    return loss

In [25]:
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

In [40]:
model = CPNet().to(device)

In [41]:
criterion = dummy_loss
optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
best=0

In [42]:
train_time= []
train_loss = []
train_losses.append(train_loss)
tests = []
test_acc = []
test_results_exp.append(tests)
test_accs.append(test_acc)
ratio = 0
for epoch in range(1,600):
    adjust_learning_rate(optimizer, epoch)
    t1 = datetime.now()
    train_error,test_error,test_acc_this = train_icp(model,device, train_loader, optimizer,criterion,epoch,'on')
    train_loss.extend(train_error)
    tests.extend(test_error)
    test_acc.extend(test_acc_this)
    train_time.append((datetime.now()-t1).total_seconds())
    print((datetime.now()-t1).total_seconds())

Train Epoch: 1 [0/9840 (0%)]	Loss: 580.176758	Time: 0.58
Train Epoch: 1 [2560/9840 (26%)]	Loss: 19.894581	Time: 11.25
Train Epoch: 1 [5120/9840 (53%)]	Loss: 4.207623	Time: 21.60
Train Epoch: 1 [7680/9840 (79%)]	Loss: 1.792368	Time: 31.75
Train Epoch: 1 Accuracy: 0/9840 (0.00%)

39.347571
Train Epoch: 2 [0/9840 (0%)]	Loss: 1.231785	Time: 0.51
Train Epoch: 2 [2560/9840 (26%)]	Loss: 0.577175	Time: 10.59
Train Epoch: 2 [5120/9840 (53%)]	Loss: 0.415102	Time: 20.55
Train Epoch: 2 [7680/9840 (79%)]	Loss: 0.280978	Time: 30.57
Train Epoch: 2 Accuracy: 0/9840 (0.00%)

38.072406
Train Epoch: 3 [0/9840 (0%)]	Loss: 0.191989	Time: 0.50
Train Epoch: 3 [2560/9840 (26%)]	Loss: 0.079816	Time: 10.47
Train Epoch: 3 [5120/9840 (53%)]	Loss: 0.044045	Time: 20.38
Train Epoch: 3 [7680/9840 (79%)]	Loss: 0.025522	Time: 30.19
Train Epoch: 3 Accuracy: 0/9840 (0.00%)

37.519726
Train Epoch: 4 [0/9840 (0%)]	Loss: 0.023181	Time: 0.50
Train Epoch: 4 [2560/9840 (26%)]	Loss: 0.040799	Time: 10.28
Train Epoch: 4 [5120/984

Train Epoch: 29 [5120/9840 (53%)]	Loss: 0.000006	Time: 19.30
Train Epoch: 29 [7680/9840 (79%)]	Loss: 0.000022	Time: 28.69
Train Epoch: 29 Accuracy: 0/9840 (0.00%)

35.7524
Train Epoch: 30 [0/9840 (0%)]	Loss: 0.000041	Time: 0.47
Train Epoch: 30 [2560/9840 (26%)]	Loss: 0.000045	Time: 9.87
Train Epoch: 30 [5120/9840 (53%)]	Loss: 0.000054	Time: 19.30
Train Epoch: 30 [7680/9840 (79%)]	Loss: 0.000108	Time: 28.76
Train Epoch: 30 Accuracy: 0/9840 (0.00%)

35.808145
Train Epoch: 31 [0/9840 (0%)]	Loss: 0.000002	Time: 0.46
Train Epoch: 31 [2560/9840 (26%)]	Loss: 0.000008	Time: 9.86
Train Epoch: 31 [5120/9840 (53%)]	Loss: 0.000024	Time: 19.25
Train Epoch: 31 [7680/9840 (79%)]	Loss: 0.000036	Time: 28.66
Train Epoch: 31 Accuracy: 0/9840 (0.00%)

35.730218
Train Epoch: 32 [0/9840 (0%)]	Loss: 0.000020	Time: 0.47
Train Epoch: 32 [2560/9840 (26%)]	Loss: 0.000021	Time: 9.87
Train Epoch: 32 [5120/9840 (53%)]	Loss: 0.000081	Time: 19.26
Train Epoch: 32 [7680/9840 (79%)]	Loss: 0.000072	Time: 28.67
Train Epoc

Train Epoch: 57 Accuracy: 0/9840 (0.00%)

35.517275
Train Epoch: 58 [0/9840 (0%)]	Loss: 0.000009	Time: 0.47
Train Epoch: 58 [2560/9840 (26%)]	Loss: 0.000167	Time: 9.81
Train Epoch: 58 [5120/9840 (53%)]	Loss: 0.000052	Time: 19.17
Train Epoch: 58 [7680/9840 (79%)]	Loss: 0.000097	Time: 28.54
Train Epoch: 58 Accuracy: 0/9840 (0.00%)

35.527514
Train Epoch: 59 [0/9840 (0%)]	Loss: 0.000156	Time: 0.49
Train Epoch: 59 [2560/9840 (26%)]	Loss: 0.000008	Time: 9.82
Train Epoch: 59 [5120/9840 (53%)]	Loss: 0.000101	Time: 19.11
Train Epoch: 59 [7680/9840 (79%)]	Loss: 0.000027	Time: 28.48
Train Epoch: 59 Accuracy: 0/9840 (0.00%)

35.47355
Train Epoch: 60 [0/9840 (0%)]	Loss: 0.000013	Time: 0.47
Train Epoch: 60 [2560/9840 (26%)]	Loss: 0.000015	Time: 9.78
Train Epoch: 60 [5120/9840 (53%)]	Loss: 0.000016	Time: 19.10
Train Epoch: 60 [7680/9840 (79%)]	Loss: 0.000167	Time: 28.46
Train Epoch: 60 Accuracy: 0/9840 (0.00%)

35.479712
Train Epoch: 61 [0/9840 (0%)]	Loss: 0.000073	Time: 0.48
Train Epoch: 61 [2560/9

Train Epoch: 86 [2560/9840 (26%)]	Loss: 0.000061	Time: 9.86
Train Epoch: 86 [5120/9840 (53%)]	Loss: 0.000009	Time: 19.24
Train Epoch: 86 [7680/9840 (79%)]	Loss: 0.000005	Time: 28.55
Train Epoch: 86 Accuracy: 0/9840 (0.00%)

35.552937
Train Epoch: 87 [0/9840 (0%)]	Loss: 0.000021	Time: 0.46
Train Epoch: 87 [2560/9840 (26%)]	Loss: 0.000027	Time: 9.82
Train Epoch: 87 [5120/9840 (53%)]	Loss: 0.000000	Time: 19.13
Train Epoch: 87 [7680/9840 (79%)]	Loss: 0.000009	Time: 28.45
Train Epoch: 87 Accuracy: 0/9840 (0.00%)

35.473067
Train Epoch: 88 [0/9840 (0%)]	Loss: 0.000019	Time: 0.47
Train Epoch: 88 [2560/9840 (26%)]	Loss: 0.000199	Time: 9.80
Train Epoch: 88 [5120/9840 (53%)]	Loss: 0.000017	Time: 19.13
Train Epoch: 88 [7680/9840 (79%)]	Loss: 0.000009	Time: 28.46
Train Epoch: 88 Accuracy: 0/9840 (0.00%)

35.47008
Train Epoch: 89 [0/9840 (0%)]	Loss: 0.000003	Time: 0.47
Train Epoch: 89 [2560/9840 (26%)]	Loss: 0.000001	Time: 9.79
Train Epoch: 89 [5120/9840 (53%)]	Loss: 0.000005	Time: 19.13
Train Epoc

Train Epoch: 114 [5120/9840 (53%)]	Loss: 0.000012	Time: 19.07
Train Epoch: 114 [7680/9840 (79%)]	Loss: 0.000003	Time: 28.41
Train Epoch: 114 Accuracy: 0/9840 (0.00%)

35.396639
Train Epoch: 115 [0/9840 (0%)]	Loss: 0.000002	Time: 0.46
Train Epoch: 115 [2560/9840 (26%)]	Loss: 0.000004	Time: 9.78
Train Epoch: 115 [5120/9840 (53%)]	Loss: 0.000006	Time: 19.12
Train Epoch: 115 [7680/9840 (79%)]	Loss: 0.000041	Time: 28.43
Train Epoch: 115 Accuracy: 0/9840 (0.00%)

35.388188
Train Epoch: 116 [0/9840 (0%)]	Loss: 0.000004	Time: 0.46
Train Epoch: 116 [2560/9840 (26%)]	Loss: 0.000011	Time: 9.82
Train Epoch: 116 [5120/9840 (53%)]	Loss: 0.000003	Time: 19.15
Train Epoch: 116 [7680/9840 (79%)]	Loss: 0.000010	Time: 28.48
Train Epoch: 116 Accuracy: 0/9840 (0.00%)

35.497035
Train Epoch: 117 [0/9840 (0%)]	Loss: 0.000063	Time: 0.48
Train Epoch: 117 [2560/9840 (26%)]	Loss: 0.000067	Time: 9.86
Train Epoch: 117 [5120/9840 (53%)]	Loss: 0.000019	Time: 19.16
Train Epoch: 117 [7680/9840 (79%)]	Loss: 0.000000	Tim

Train Epoch: 142 [2560/9840 (26%)]	Loss: 0.000066	Time: 9.77
Train Epoch: 142 [5120/9840 (53%)]	Loss: 0.000022	Time: 19.10
Train Epoch: 142 [7680/9840 (79%)]	Loss: 0.000003	Time: 28.48
Train Epoch: 142 Accuracy: 0/9840 (0.00%)

35.472426
Train Epoch: 143 [0/9840 (0%)]	Loss: 0.000011	Time: 0.46
Train Epoch: 143 [2560/9840 (26%)]	Loss: 0.000065	Time: 9.78
Train Epoch: 143 [5120/9840 (53%)]	Loss: 0.000025	Time: 19.13
Train Epoch: 143 [7680/9840 (79%)]	Loss: 0.000207	Time: 28.43
Train Epoch: 143 Accuracy: 0/9840 (0.00%)

35.36946
Train Epoch: 144 [0/9840 (0%)]	Loss: 0.000010	Time: 0.46
Train Epoch: 144 [2560/9840 (26%)]	Loss: 0.000008	Time: 9.77
Train Epoch: 144 [5120/9840 (53%)]	Loss: 0.000004	Time: 19.08
Train Epoch: 144 [7680/9840 (79%)]	Loss: 0.000007	Time: 28.37
Train Epoch: 144 Accuracy: 0/9840 (0.00%)

35.332554
Train Epoch: 145 [0/9840 (0%)]	Loss: 0.000001	Time: 0.48
Train Epoch: 145 [2560/9840 (26%)]	Loss: 0.000024	Time: 9.78
Train Epoch: 145 [5120/9840 (53%)]	Loss: 0.000061	Time:

KeyboardInterrupt: 

In [29]:
for batch_idx, (src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba) in enumerate(train_loader):
    break

In [34]:
translation_ab.shape

torch.Size([128, 3])